In [1]:
# !pip install http://download.pytorch.org/whl/cu90/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
# !pip install torchvision
# !pip install tqdm

ERROR: torch-1.0.0-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.
  Using cached torchvision-0.12.0-cp38-cp38-macosx_10_9_x86_64.whl (1.3 MB)
  Using cached torch-1.11.0-cp38-none-macosx_10_9_x86_64.whl (129.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.2
    Uninstalling torch-1.10.2:
      Successfully uninstalled torch-1.10.2
